In [1]:
import time
import requests
import json
import pandas as pd
import os

In [13]:
df = pd.read_csv('processed_to_geo.csv')
print(type(df))
print(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
(12480780, 16)


,Unnamed: 0,Id,Date_mutation,Valeur_fonciere,No_voie,Type_de_voie,Voie,Code_postal,Commune,Code_departement,Type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,Prix_par_m_bati,Prix_par_m_terrain
0,0,0,2019-04-01,37220,26,RUE,DE MONTHOLON,1000,BOURG-EN-BRESSE,1,Appartement,20.0,1.0,NaN,1861.000000,0.000000
1,1,1,2019-04-01,185100,22,RUE,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,1,Appartement,62.0,3.0,NaN,2985.483871,0.000000
2,2,2,2019-04-01,185100,22,RUE,GEN DELESTRAINT,1000,BOURG-EN-BRESSE,1,Dépendance,0.0,0.0,NaN,0.000000,0.000000
3,3,3,2019-08-01,209000,3,RUE,DES CHAMPAGNES,1160,PRIAY,1,Maison,90.0,4.0,940.0,2322.222222,222.340426
4,4,4,2019-07-01,134900,5,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,1,Maison,101.0,5.0,490.0,1335.643564,275.306122


In [14]:
columns = ['No_voie', 'Type_de_voie', 'Voie', 'Code_postal', 'Commune']
print(columns)

['No_voie', 'Type_de_voie', 'Voie', 'Code_postal', 'Commune']


In [15]:
if 'adresse' not in df:
    addresses = df[columns].apply(lambda cols: ' '.join(cols.values.astype(str)), axis = 1)
else:
    addresses = df['adresse']

print(type(addresses))

<class 'pandas.core.series.Series'>


In [16]:
addresses_df = addresses.drop_duplicates().to_frame(name='adresse')
print(type(addresses_df))
print(addresses_df.shape)
print(addresses_df.head())

<class 'pandas.core.frame.DataFrame'>
(4258813, 1)
                                       adresse
0     26 RUE DE MONTHOLON 1000 BOURG-EN-BRESSE
1  22 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE
3              3 RUE DES CHAMPAGNES 1160 PRIAY
4   5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS
5         165 ALL DES LIBELLULES 1340 ATTIGNAT


In [17]:
addresses_df.to_csv('addresses.csv', index=False)

In [18]:
addresses_df = pd.read_csv('addresses.csv')
addresses_df.index.rename('index', inplace=True)
print(type(addresses_df))
print(addresses_df.shape)
print(addresses_df.head())

<class 'pandas.core.frame.DataFrame'>
(4258813, 1)
                                           adresse
index                                             
0         26 RUE DE MONTHOLON 1000 BOURG-EN-BRESSE
1      22 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE
2                  3 RUE DES CHAMPAGNES 1160 PRIAY
3       5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS
4             165 ALL DES LIBELLULES 1340 ATTIGNAT


In [21]:
if not os.path.exists('data'):
    os.makedirs('data')

batch_size = 100000
total = addresses_df.shape[0]
total_batches = len(range(0,total,batch_size))
for i, start in enumerate(range(0,total,batch_size)):
    end = min(start+batch_size,total)
    sub_addresses_df = addresses_df.iloc[start:end]
    sub_addresses_df.to_csv(f'data/addresses_{i}.csv', index=False)
    

In [22]:
url = 'https://api-adresse.data.gouv.fr/search/csv/'
# for i in range(0,total,batch_size):
for i, start in enumerate(range(0,total,batch_size)):
    t = time.time()
    print(f'starting batch {i+1}/{total_batches}')
    with open(f'data/addresses_{i}.csv','rb') as input:
        r = requests.post(url,files={'data':('data.csv',input)})
        with open(f'data/addresses_{i}_result.csv','wb') as output:
            output.write(r.content)
    print(f'finished batch {i+1}/{total_batches} in {time.time() - t} sec')

starting batch 1/4
finished batch 1/4 in 5.972693920135498 sec
starting batch 2/4
finished batch 2/4 in 8.071311235427856 sec
starting batch 3/4
finished batch 3/4 in 8.050302028656006 sec
starting batch 4/4
finished batch 4/4 in 5.150148868560791 sec


In [33]:
def adresse_to_lon_lat(adresse):
    url = "https://api-adresse.data.gouv.fr/search"
    response = requests.get(url, params = {'q': adresse, 'limit': '1'})
    result = response.json()
    longitude = result['features'][0]['geometry']['coordinates'][0]
    latitude = result['features'][0]['geometry']['coordinates'][1]
    return (longitude, latitude)

In [31]:
addresses_df_empty = addresses_df
addresses_df_empty = addresses_df_empty[addresses_df_empty['score'].isna()]
print(type(addresses_df_empty))
print(addresses_df_empty.shape)
print(addresses_df_empty.head())

(4258813, 5)
                                       adresse  longitude  latitude  score  \
0     26 RUE DE MONTHOLON 1000 BOURG-EN-BRESSE        NaN       NaN    NaN   
1  22 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE        NaN       NaN    NaN   
2              3 RUE DES CHAMPAGNES 1160 PRIAY        NaN       NaN    NaN   
3   5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS        NaN       NaN    NaN   
4         165 ALL DES LIBELLULES 1340 ATTIGNAT        NaN       NaN    NaN   

   error  
0  False  
1  False  
2  False  
3  False  
4  False  


In [ ]:
type(addresses)

In [5]:
df_empty = df
df_empty = df_empty[df_empty['longitude'] == '']
df_empty = df_empty[df_empty['latitude'] == '']

while not df_empty.empty:
    adresses = pd.Series(df_empty.index)
    adresses.drop_duplicates(inplace=True)

    t = time.time()
    for adresse in adresses[0:100]:
        lon, lat = adresse_to_lon_lat(adresse)
        df['longitude'][adresse] = lon
        df['latitude'][adresse] = lat
        df_empty.drop(index=adresse, inplace=True)

    print(time.time()-t, "secondes")

    df.to_csv('processed_to_geo.csv')

    print("shape of df_empty remaining ", df_empty.shape)

KeyboardInterrupt: 

In [6]:
df_discovery = pd.read_csv('processed_to_geo.csv')
df_discovery[1000:].head()

,adresse,Unnamed: 0,Id,Date_mutation,Valeur_fonciere,No_voie,Type_de_voie,Voie,Code_postal,Commune,Code_departement,Type_local,Surface_reelle_bati,Nombre_pieces_principales,Surface_terrain,Prix_par_m_bati,Prix_par_m_terrain,longitude,latitude
1000,0 OZAN 1190 OZAN,1137,1137,2019-11-02,130000,0,,OZAN,1190,OZAN,1,NaN,NaN,0.0,798.0,0.000000,162.907268,NaN,NaN
1001,0 OZAN 1190 OZAN,1138,1138,2019-11-02,130000,0,,OZAN,1190,OZAN,1,NaN,NaN,0.0,219.0,0.000000,593.607306,NaN,NaN
1002,684 GRANDE ROUTE 1190 OZAN,1139,1139,2019-11-02,130000,684,,GRANDE ROUTE,1190,OZAN,1,Maison,72.0,2.0,730.0,1805.555556,178.082192,NaN,NaN
1003,0 OZAN 1190 OZAN,1140,1140,2019-11-02,130000,0,,OZAN,1190,OZAN,1,NaN,NaN,0.0,54.0,0.000000,2407.407407,NaN,NaN
1004,455 RTE DE SAINT JEAN 1290 SAINT-JEAN-SUR-VEYLE,1141,1141,2019-02-15,143800,455,RTE,DE SAINT JEAN,1290,SAINT-JEAN-SUR-VEYLE,1,Maison,74.0,3.0,1054.0,1943.243243,136.432638,NaN,NaN
